In [1]:
import cv2  # Import the OpenCV library for video processing.
import pyttsx3  # Import pyttsx3 for text-to-speech functionality.
import threading  # Import threading for running the alarm message in a separate thread.

# Create a list to store the motion detection status for the current and previous frame.
status_list = [None, None]

# Initialize the text-to-speech engine.
alarm_sound = pyttsx3.init()

# Get the available voices and set the preferred voice and speech rate.
voices = alarm_sound.getProperty('voices')
alarm_sound.setProperty('voice', voices[0].id)
alarm_sound.setProperty('rate', 150)

# Define a function for the voice alarm.
def voice_alarm(alarm_sound):
    alarm_sound.say("Thief Detected")  # Set the text to be spoken.
    alarm_sound.runAndWait()  # Execute the voice alarm.

# Open a video file for processing. In this case, "theifs.mp4".
video = cv2.VideoCapture("theifs.mp4")

# Initialize a variable to store the initial frame for motion detection.
initial_frame = None

# Start a loop to process video frames.
while True:
    # Read the next frame from the video.
    check, frame = video.read()
    
    # Initialize a variable to count the number of objects in motion.
    status = 0

    # Convert the current frame to grayscale.
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur and smoothing to the grayscale frame to reduce noise.
    gray_frame = cv2.GaussianBlur(gray_frame, (25, 25), 0)
    blur_frame = cv2.blur(gray_frame, (5, 5))
    
    # Check if it's the initial frame (the first frame).
    if initial_frame is None:
        initial_frame = blur_frame
        continue

    # Calculate the absolute difference between the initial frame and the current frame.
    delta_frame = cv2.absdiff(initial_frame, blur_frame)

    # Apply a threshold to the difference frame to create a binary image that highlights changes.
    threshold_frame = cv2.threshold(delta_frame, 35, 255, cv2.THRESH_BINARY)[1]

    # Find contours in the binary image to detect objects.
    (contours, _) = cv2.findContours(threshold_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop through the detected contours.
    for c in contours:
        # If the contour area is less than 5000, it's considered noise and skipped.
        if cv2.contourArea(c) < 5000:
            continue
        # Increment the status count for detected objects.
        status = status + 1
        # Get the coordinates and dimensions of the bounding rectangle around the object.
        (x, y, w, h) = cv2.boundingRect(c)
        # Draw a green rectangle around the detected object on the current frame.
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)

    # Append the current status to the status list.
    status_list.append(status)

    # Check if the current status is 1 (indicating motion) and the previous status was 0 (indicating no motion).
    if status_list[-1] >= 1 and status_list[-2] == 0:
        # Start a separate thread to run the voice alarm function.
        alarm = threading.Thread(target=voice_alarm, args=(alarm_sound,))
        alarm.start()

    # Display the current frame with rectangles drawn around detected objects.
    cv2.imshow('motion detector', frame)

    # Check for the 'q' key press to exit the loop.
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

# Stop the voice alarm and release the video file.
alarm_sound.stop()
video.release()

# Close all OpenCV windows and finish the program.
cv2.destroyAllWindows()


Exception in thread Thread-7 (voice_alarm):
Traceback (most recent call last):
  File "C:\Users\brish\anaconda3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\brish\anaconda3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\brish\AppData\Local\Temp\ipykernel_18664\3413035849.py", line 19, in voice_alarm
  File "C:\Users\brish\anaconda3\lib\site-packages\pyttsx3\engine.py", line 177, in runAndWait
    raise RuntimeError('run loop already started')
RuntimeError: run loop already started
Exception in thread Thread-8 (voice_alarm):
Traceback (most recent call last):
  File "C:\Users\brish\anaconda3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\brish\anaconda3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\brish\AppData\Local\Temp\ipykernel_18664\3413035849.py", line 19, in voice_alarm
  File "C:\Users\brish\anaconda3\li

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
